In [42]:
import pandas as pd
import json
import requests
import re

### Read Data

In [2]:
URL = 'https://api.census.gov/data/2012/acs5/variables.json'

In [3]:
value_dict = requests.get(URL).json()

### Build race information

In [44]:
race_dict = {'A': 'White Alone',
'B': 'Black or African American Alone',
'C': 'American Indian and Alaska Native Alone',
'D': 'Asian Alone',
'E': 'Native Hawaiian and Other Pacific Islander Alone',
'F': 'Some Other Race Alone',
'G': 'Two or More Races',
'H': 'White Alone, Not Hispanic or Latino',
'I': 'Hispanic or Latino'}

#### Build dataframe with information from each variable

The dictionary of variables is built as a nested dictionary with the following structure:

- Concept: The name of the variable
- Label: The specific socioeconomic cuts that data belongs to
- predicateType: The type of variable

In [27]:
value_dict['variables']['B17010B_030E']

{'concept': 'B17010B.  Poverty Status of Families by Family Type by Presence of Related Children Under 18 Years by Age (Black Alone Hhldr)',
 'group': 'N/A',
 'label': 'Income in the past 12 months at or above poverty level:!!Other family:!!Male householder, no wife present:',
 'limit': 0,
 'predicateType': 'int'}

Get the set of values for group, limit and predicateType

In [35]:
set_type = set()
set_group = set()
set_limit = set()
for i in value_dict['variables'].keys():
    try: 
        set_type.add(value_dict['variables'][i]['predicateType'])
        set_limit.add(value_dict['variables'][i]['limit'])
        set_group.add(value_dict['variables'][i]['group'])
    except: 
        pass
    if value_dict['variables'][i]['group'] != 'N/A':
        print(value_dict['variables'][i])
        break

In [36]:
set_type

{'fips-for', 'fips-in', 'int', 'string'}

In [37]:
set_group

{'N/A'}

In [38]:
set_limit

{0}

In [ ]:
re.match('')

### Build dataframe from dictionary

In [53]:
def get_race(string):
    code = string.split("_")
    if len(code) >= 2:
        char = code[0][-1]
        try:
            race_value = race_dict[char]
        except:
            race_value = ''
    else:
        race_value = ''
    
    return race_value

In [60]:
counter = 0
result_df = { 'id': [], 'concept': [], 'label': [], 'race' : [], 'type' : []}

for i in value_dict['variables'].keys():
    counter += 1
#     print(i, value_dict['variables'][i]['concept'], value_dict['variables'][i]['label'])
    
    variable_id = i
    variable_concept = value_dict['variables'][variable_id]['concept']
    variable_label = value_dict['variables'][variable_id]['label'].split(':!!')
    variable_race = get_race(variable_id)
    
    try:
        variable_type = value_dict['variables'][variable_id]['predicateType']
    except:
        variable_type = ''

    result_df['id'].append(variable_id)
    result_df['concept'].append(variable_concept)
    result_df['label'].append(variable_label)
    result_df['race'].append(variable_race)
    result_df['type'].append(variable_type)

Create dataframe and a dummy indicating if dummy is for margin of error

In [62]:
compiled_results = pd.DataFrame(result_df)


#### Split the information from each label
The information of each label is sometimes in the following format where there are different subgroups represented

In [41]:
value_dict['variables']['B17010B_030E']['label']

'Income in the past 12 months at or above poverty level:!!Other family:!!Male householder, no wife present:'

The code splits based on the string ":!!" and creates a column for each one of those values

In [63]:
for i in range(1,6):
    compiled_results['label_' + str(i)] = compiled_results.label.apply(  lambda x : x[i - 1] if len(x) >= i else None)

Create dummy indicating ifvariable is a margin of error

In [67]:
compiled_results['margin_error_dummy'] = compiled_results.label_1.apply( lambda x : 'Margin' in x)
compiled_results.drop(['label'], axis = 1, inplace = True)

In [68]:
compiled_results.to_csv('dictionary_table_v2.csv')

In [69]:
compiled_results.head()

,concept,id,race,type,label_1,label_2,label_3,label_4,label_5,margin_error_dummy
0,B24125. Detailed Occupation for the Full-Time...,B24125_425M,,int,Margin Of Error For!!Computer control programm...,None,None,None,None,True
1,B17020G. POVERTY STATUS IN THE PAST 12 MONTHS...,B17020G_004M,Two or More Races,int,Margin Of Error For!!Income in the past 12 mon...,5 years,None,None,None,True
2,B20005B. Sex by Work Experience by Earnings f...,B20005B_061E,Black or African American Alone,int,Female,"Worked full-time, year-round in the past 12 mo...",With earnings,"$20,000 to $22,499",None,False
3,B20005G. Sex by Work Experience by Earnings f...,B20005G_058E,Two or More Races,int,Female,"Worked full-time, year-round in the past 12 mo...",With earnings,"$12,500 to $14,999",None,False
4,B07010PR. Geographical Mobility in the Past Y...,B07010PR_015M,,int,Margin of Error for!!Same house 1 year ago,With income,"$1 to $9,999 or loss",None,None,True


### Choose results for black population

In [71]:
black_population = compiled_results[compiled_results.race == 'Black or African American Alone']

In [82]:
len(black_population)

1202

### Choose results for blended population
It will select the same variables than the black population

In [77]:
blended_pop_index = black_population.id.apply(lambda x : x.split("_")[0][:-1] + "_" +  x.split("_")[1])

In [98]:
blended_population = blended_pop_index.to_frame().merge(compiled_results, how = 'inner', on = 'id')

In [99]:
len(blended_population)

976

In [100]:
blended_population

,id,concept,race,type,label_1,label_2,label_3,label_4,label_5,margin_error_dummy
0,B20005_061E,B20005. Sex by Work Experience by Earnings fo...,,int,Female,"Worked full-time, year-round in the past 12 mo...",With earnings,"$20,000 to $22,499",None,False
1,B19037_035M,B19037. Age of Householder by Household Income,,int,Margin of Error for!!Householder 25 to 44 years,"$200,000 or more",None,None,None,True
2,B20005_049M,B20005. Sex by Work Experience by Earnings fo...,,int,Margin of Error for!!Female:,None,None,None,None,True
3,B20005_080M,B20005. Sex by Work Experience by Earnings fo...,,int,Margin of Error for!!Female,Other,With earnings,"$10,000 to $12,499",None,True
4,B19037_009M,B19037. Age of Householder by Household Income,,int,Margin of Error for!!Householder under 25 years,"$35,000 to $39,999",None,None,None,True
5,B19037_043M,B19037. Age of Householder by Household Income,,int,Margin of Error for!!Householder 45 to 64 years,"$35,000 to $39,999",None,None,None,True
6,B20005_037M,B20005. Sex by Work Experience by Earnings fo...,,int,Margin of Error for!!Male,Other,With earnings,"$20,000 to $22,499",None,True
7,B14007_012E,B14007. School Enrollment by Level of School ...,,int,Enrolled in school,Enrolled in grade 8,None,None,None,False
8,B01001_027M,B01001. Sex by Age,,int,Margin Of Error For!!Female,Under 5 years,None,None,None,True
9,B25003_002E,B25003. Tenure,,int,Owner occupied,None,None,None,None,False


In [101]:
total_population = black_population.append(blended_population)

### Save results

In [102]:
total_population.to_csv('all_population.csv')
total_population.id.values.tolist()

['B20005B_061E',
 'B19037B_035M',
 'B20005B_049M',
 'B20005B_080M',
 'B08105B_005E',
 'B19037B_009M',
 'B19037B_043M',
 'B20005B_037M',
 'B14007B_012E',
 'B01001B_027M',
 'B25003B_002E',
 'C23002B_009M',
 'B19037B_013M',
 'B08105B_003E',
 'B01001B_025E',
 'B05003B_017E',
 'B17010B_002E',
 'B16005B_002M',
 'B17010B_009E',
 'C15002B_010E',
 'B14007B_002M',
 'C27001B_010M',
 'B19037B_007M',
 'B17020B_006E',
 'B17001B_036E',
 'B19037B_005E',
 'B05003B_011M',
 'B01001B_014M',
 'B17010B_017M',
 'B05003B_001M',
 'B08505B_006M',
 'B20005B_093M',
 'B19001B_010E',
 'B08505B_002E',
 'C23002B_008E',
 'B08105B_004M',
 'B14007B_017E',
 'B05003B_006M',
 'B01001B_018E',
 'B01002B_001E',
 'B20005B_057E',
 'B17001B_031E',
 'B17001B_017M',
 'B16005B_004E',
 'B05003B_018M',
 'B01001B_031E',
 'B20005B_078M',
 'B01001B_003M',
 'B19037B_050E',
 'B05003B_007E',
 'B19001B_004M',
 'B08505B_007E',
 'C21001B_003E',
 'B07004B_003E',
 'B13002B_004E',
 'B19037B_020M',
 'B19001B_011M',
 'B17020B_017M',
 'B20005B_019M